In [1]:
!pip install Pydub openai-whisper openai noisereduce -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 45.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.1/378.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [2]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.effects import low_pass_filter, high_pass_filter, compress_dynamic_range

import whisper
from openai import OpenAI

import librosa

import soundfile as sf

import tempfile

from google.colab import drive
import os

import re
import pandas as pd

import noisereduce as nr

In [ ]:
# montando o drive e pegando o diretorio com os áudios
drive.mount('/content/drive')
diretorio = '/content/drive/MyDrive/Challenge TOTVS Amostra de Dados v2/Amostra de Dados'

arquivos = os.listdir(diretorio)

# pegando as instruções do chat got por meio de um txt no drive, deixando o
# código mais limpo (as instruções estão na documentação)
with open('/content/drive/MyDrive/Challenge TOTVS Amostra de Dados v2/instrucoes.txt', 'r') as file:
    instrucoes_gpt = file.read()

# pegando a api por um txt no drive por questões de segurança
with open('/content/drive/MyDrive/Challenge TOTVS Amostra de Dados v2/api_key.txt', 'r') as file:
    api_key = file.read()

# baixando o modelo medium do whisper para transcrição ed inicializando as listas
model = whisper.load_model('medium')
dfs, respostas = [], []

# criando a função de processamento de áudios, que recebe o áudio do drive como parâmetro
def processar_audio(caminho_arquivo):

    # carregando os áudios e retirando o ruído
    y, sr = librosa.load(caminho_arquivo, sr=None)
    noise_sample = y[:int(sr)] # amostra do ruído
    y_reduced_noise = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample, prop_decrease=0.8)

    # criando um arquivo temporário para continuar os processos
    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_wav:
        sf.write(temp_wav.name, y_reduced_noise, sr)
        temp_wav_path = temp_wav.name

    # carregando o arquivo temporário e passando filtros nas freqûencias do áudio para melhorá-lo
    audio = AudioSegment.from_wav(temp_wav_path)
    audio_low = low_pass_filter(audio, cutoff=4000)
    audio_high = high_pass_filter(audio_low, cutoff=80)

    # aumentando o volume do áudio
    audio_amplificado = audio_high.apply_gain(7.0)

    #retornando o áudio final
    return audio_amplificado


# definindo a função que normaliza as respostas do GPT
def limpar_resposta(resposta):
    resposta = re.sub(r'^[-\*\s]+', '', resposta, flags=re.MULTILINE).strip()
    return re.sub(r'\n\s*\n', '\n', resposta).split('\n')

# definindo a função que transforma as respostas do GPT em dataframe
def processar_resposta(linhas, arquivo):

    # inicializando as listas que virarão colunas
    topicos, notas, justificativas = [], [], []

    # fazendo a separção das respostas por linhas, separando tópicos, notas e justificativas por : e -
    for linha in linhas:
        topico, detalhes = linha.split(': ', 1) #separando os tópicos
        if ' - ' in detalhes:
            nota, justificativa = detalhes.split(' - ', 1) #separando as notas das justificativas
        else:
            nota, justificativa = detalhes, ''
        topicos.append(topico)
        notas.append(nota)
        justificativas.append(justificativa)

    # retornando o DF utilizando as listas e também o nome do arquivo como ID
    return pd.DataFrame({
        'Tópico': topicos,
        'Nota': notas,
        'Justificativa': justificativas,
        'ID': arquivo
    })

# utilizando um loop dentro da pasta de áudios
for arquivo in arquivos:
    if arquivo.endswith('.wav'):
        caminho_arquivo = os.path.join(diretorio, arquivo)

        # realizando o processamento do áudio
        try:
            audio_amplificado = processar_audio(caminho_arquivo)

            # criando um arquivo temporário para que a transcrição seja feita
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_final_wav:
                audio_amplificado.export(temp_final_wav.name, format='wav')
                temp_final_wav_path = temp_final_wav.name

            result = model.transcribe(temp_final_wav_path)

        # desfazendo o arquivo temporário
        finally:
            os.unlink(temp_final_wav_path)

        # criando o texto do gpt que pega as instruções junto do resultado da transcrição
        texto_gpt = instrucoes_gpt + result['text'].replace('\n', ' ')

        # acionando a API
        client = OpenAI(api_key=api_key)

        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": texto_gpt}],
            model="gpt-4o-2024-08-06",
        )

        # limpando a resposta do gpt, criando o DF e adiconando na lista
        resposta = chat_completion.choices[0].message.content
        linhas = limpar_resposta(resposta)
        df = processar_resposta(linhas, arquivo)
        dfs.append(df)

# concatenando todos os DFs e transformando em csv
df_final = pd.concat(dfs, ignore_index=True)

# Dicionário de mapeamento de números por extenso para algarismos
numeros_extenso_para_algarismos = {
    'Um': 1,
    'Dois': 2,
    'Três': 3,
    'Quatro': 4,
    'Cinco': 5,
    'Seis': 6,
    'Sete': 7,
    'Oito': 8,
    'Nove': 9,
    'Dez': 10
}

# Substituindo os números por extenso pelos algarismos na coluna 'Nota'
df_final['Nota'] = df_final['Nota'].replace(numeros_extenso_para_algarismos)

# retirando os NaN's da coluna justificativa na parte de "Notas",
# "pontos adicionais(quando respondidos com 'não')" e "classificação do cliente"
df_final['Justificativa'] = df_final['Justificativa'].fillna('Sem justificativa')

# transformando em csv e armazenando no Drive
df_final.to_csv('/content/drive/MyDrive/Challenge TOTVS Amostra de Dados v2s/respostas_gpt_4.csv', index=False)